<h3>VVI</h3>

In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from pydantic import BaseModel,Field
from langchain_core.tools import Tool, StructuredTool
from langchain import hub
from langchain.agents import (create_openai_tools_agent,create_tool_calling_agent,
                        AgentExecutor,create_openai_functions_agent)

In [3]:
load_dotenv(find_dotenv("../.env"))

True

In [4]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-4o-mini")

In [5]:
llmMathChain=LLMMathChain.from_llm(llm=llmOpenAI,verbose=True)

In [6]:
llmMathChain.invoke(input={"What is 34 to the power 2 plus 43^2?"})



> Entering new LLMMathChain chain...
{'What is 34 to the power 2 plus 43^2?'}```text
34**2 + 43**2
```
...numexpr.evaluate("34**2 + 43**2")...

Answer: 3005
> Finished chain.


{'question': {'What is 34 to the power 2 plus 43^2?'},
 'answer': 'Answer: 3005'}

In [7]:
# Define Tools which provides 1) The nth prime number and 2) LLMMathChain to act as the calculator

primes={998:7901, 999: 7907, 1000: 7919}

##########----------Class Which utilizes in the MathsChain----------############
class CalculatorInput(BaseModel):
    question:str=Field(description="Question to ask to the Calculator")

##########-----------Class for Finding the llm_math_chain ---------------############

def getPrime(n: int, primes:dict=primes) ->str:
    return str(primes.get(int(n)))

async def agetPrime(n: int, primes:dict=primes) ->str:
    return str(primes.get(int(n)))

class PrimeInput(BaseModel):
    n:int=Field()

In [8]:
tools=[
    Tool(
        name="GetPrime",
        func=getPrime,
        description="A Tool that returns the `n`th prime number",
        args_schema=PrimeInput,  # the input
        coroutine=agetPrime
    ),
    Tool(
        name="Calculator",
        description="Useful when you would need to compute mathematical expressions",
        func=llmMathChain.invoke,
        args_schema=CalculatorInput,  # the input
        coroutine=llmMathChain.ainvoke
    )
]


In [9]:
def isPrime(n:int) -> bool:
    if n<=1 or (n%2==0 and n>2):
        return False
    else:
        for i in range(3,int(n**0.5)+1,2):
            if n%i==0:
                return False
        return True

In [10]:
prompt=hub.pull(owner_repo_commit="hwchase17/openai-tools-agent")

In [11]:
agent=create_openai_tools_agent(llm=llmOpenAI,tools=tools, prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [12]:
question="What is the product of the 998th, 999th and 1000th prime numbers?"

In [13]:
agentExecutor.invoke(input={"input":question})



> Entering new AgentExecutor chain...

Invoking: `GetPrime` with `{'n': 998}`


7901
Invoking: `GetPrime` with `{'n': 999}`


7907
Invoking: `GetPrime` with `{'n': 1000}`


7919
Invoking: `Calculator` with `{'question': '7901 * 7907 * 7919'}`




> Entering new LLMMathChain chain...
7901 * 7907 * 7919```text
7901 * 7907 * 7919
```
...numexpr.evaluate("7901 * 7907 * 7919")...

Answer: 494725326233
> Finished chain.
{'question': '7901 * 7907 * 7919', 'answer': 'Answer: 494725326233'}The product of the 998th, 999th, and 1000th prime numbers (7901, 7907, and 7919) is 494,725,326,233.

> Finished chain.


{'input': 'What is the product of the 998th, 999th and 1000th prime numbers?',
 'output': 'The product of the 998th, 999th, and 1000th prime numbers (7901, 7907, and 7919) is 494,725,326,233.'}

In [14]:
for step in agentExecutor.iter(inputs={"input":question}):
    print(step)
    print("--------")



> Entering new None chain...

Invoking: `GetPrime` with `{'n': 998}`


7901
Invoking: `GetPrime` with `{'n': 999}`


7907
Invoking: `GetPrime` with `{'n': 1000}`


7919{'intermediate_step': [(ToolAgentAction(tool='GetPrime', tool_input={'n': 998}, log="\nInvoking: `GetPrime` with `{'n': 998}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_71rzKstm2n2EOjOKEJole7KX', 'function': {'arguments': '{"n": 998}', 'name': 'GetPrime'}, 'type': 'function'}, {'index': 1, 'id': 'call_Z4eUPkzJFRl0uJLlzdYUqp2s', 'function': {'arguments': '{"n": 999}', 'name': 'GetPrime'}, 'type': 'function'}, {'index': 2, 'id': 'call_LI5LDvq1Eo9vn6MzcPmyFizV', 'function': {'arguments': '{"n": 1000}', 'name': 'GetPrime'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c'}, id='run--f841b761-dda0-4d93-98a8-92c32934bb7c', tool_calls=[{'name': 'GetPrime', 'args'

In [15]:
for step in agentExecutor.iter(inputs={"input":question}):
    if step.get('intermediate_step'):
        output=step['intermediate_step']
        action,value=output[0]
        if action.tool=="GetPrime":
            value=int(value)
            print(f"Checking whether {value} is Prime")
            assert isPrime(value)
        # # Ask users if they would want to continue
        _continue=input("Should the agent continue (Y/n)") or "Y"
        if _continue.lower()!="y":
            break
        print("--------")



> Entering new None chain...

Invoking: `GetPrime` with `{'n': 998}`


7901
Invoking: `GetPrime` with `{'n': 999}`


7907
Invoking: `GetPrime` with `{'n': 1000}`


7919Checking whether 7901 is Prime
--------

Invoking: `Calculator` with `{'question': '7901 * 7907 * 7919'}`




> Entering new LLMMathChain chain...
7901 * 7907 * 7919```text
7901 * 7907 * 7919
```
...numexpr.evaluate("7901 * 7907 * 7919")...

Answer: 494725326233
> Finished chain.
{'question': '7901 * 7907 * 7919', 'answer': 'Answer: 494725326233'}--------
The product of the 998th, 999th, and 1000th prime numbers (7901, 7907, and 7919) is 494,725,326,233.

> Finished chain.


<h3> Doing Similar stuffs with Stream</h3>

In [33]:
for step in agentExecutor.stream(input={"input":question}):
    if step.get('steps'):
        output=step['steps']  # Replace intermediate_step with step
        action,value=output[0]
        action=action[1]  # Extra Step
        if action.tool=="GetPrime":
            value=value[1]  # Extra Step
            value=int(value)
            print(f"Checking whether {value} is Prime")
            assert isPrime(value)
        # # Ask users if they would want to continue
        _continue=input("Should the agent continue (Y/n)") or "Y"
        if _continue.lower()!="y":
            break
        print("--------")



> Entering new None chain...

Invoking: `GetPrime` with `{'n': 998}`


7901Checking whether 7901 is Prime
